In [14]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Your CRUD module import (update if different)
from CRUD_Python_Module import AnimalShelter

# Connect to the database
username = "aacuser"
password = "Gengar00"
db = AnimalShelter()

# Initial data load
df = pd.DataFrame.from_records(db.read({}))
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)


# Load the logo
image_filename = 'Grazioso Salvare Logo.png'  # Ensure this file is in your working directory
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Setup the app
app = JupyterDash(__name__)
JupyterDash.infer_jupyter_proxy_config()

# Layout
app.layout = html.Div([
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '200px'})),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(html.B(html.H2('Anthony Procacina 10/19/25'))),
    html.Hr(),

    # Button filters
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Button(id='button-one', n_clicks=0, children='Water Rescue'),
        html.Button(id='button-two', n_clicks=0, children='Mountain or Wilderness Rescue'),
        html.Button(id='button-three', n_clicks=0, children='Disaster Rescue or Individual Tracking'),
        html.Button(id='button-four', n_clicks=0, children='Reset')
    ]),
    html.Hr(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=100,
        style_table={'height': '400px', 'overflowY': 'auto', 'overflowX': 'auto'},
        style_header={'backgroundColor': 'rgb(240,230,230)', 'fontWeight': 'bold'},
        style_data={'whiteSpace': 'normal', 'height': 'auto'},
        tooltip={i: {'value': i, 'use_with': 'both'} for i in df.columns},
        tooltip_delay=0,
        tooltip_duration=None,
        sort_action='native',
        sort_mode='multi',
        filter_action='native',
        editable=False,
        column_selectable='multi',
        row_selectable='single',
        row_deletable=False,
        selected_rows=[]
    ),

    html.Br(),
    html.Hr(),

    # Graph and Map
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Button filter callback
@app.callback(
    Output('datatable-id', 'data'),
    [
        Input('button-one', 'n_clicks'),
        Input('button-two', 'n_clicks'),
        Input('button-three', 'n_clicks'),
        Input('button-four', 'n_clicks')
    ]
)
def update_dashboard(button1, button2, button3, button4):
    from dash import callback_context
    ctx = callback_context

    # Default query: return all documents
    query = {}

    if ctx.triggered:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]

        if button_id == 'button-one':  # Water Rescue
            query = {
                '$and': [
                    {'$or': [
                        {'breed': 'Labrador Retriever Mix'},
                        {'breed': 'Chesapeake Bay Retriever'},
                        {'breed': 'Newfoundland'}
                    ]},
                    {'sex_upon_outcome': 'Intact Female'},
                    {'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
                ]
            }

        elif button_id == 'button-two':  # Mountain/Wilderness
            query = {
                '$and': [
                    {'$or': [
                        {'breed': 'German Shepherd'},
                        {'breed': 'Alaskan Malamute'},
                        {'breed': 'Old English Sheepdog'},
                        {'breed': 'Siberian Husky'},
                        {'breed': 'Rottweiler'}
                    ]},
                    {'sex_upon_outcome': 'Intact Male'},
                    {'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
                ]
            }

        elif button_id == 'button-three':  # Disaster Rescue
            query = {
                '$and': [
                    {'$or': [
                        {'breed': 'Doberman Pinscher'},
                        {'breed': 'German Shepherd'},
                        {'breed': 'Golden Retriever'},
                        {'breed': 'Bloodhound'},
                        {'breed': 'Rottweiler'}
                    ]},
                    {'sex_upon_outcome': 'Intact Male'},
                    {'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}}
                ]
            }

        # Else button-four = Reset → default query (empty)

    # Query the database
    try:
        df_out = pd.DataFrame.from_records(db.read(query))
        if '_id' in df_out.columns:
            df_out.drop(columns=['_id'], inplace=True)
        return df_out.to_dict('records')
    except Exception as e:
        print("❌ Error in update_dashboard:", e)
        return []




# Graph callback
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return html.P("No data to display")
    
    df = pd.DataFrame.from_dict(viewData)
    breed_counts = df['breed'].value_counts().reset_index()
    breed_counts.columns = ['breed', 'count']

    return [
        dcc.Graph(
            figure=px.pie(breed_counts, values='count', names='breed', title='Percentage of breeds available')
        )
    ]


# Table highlight callback
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]



# Map update callback
@app.callback(
    Output('map-id', "children"),
    [
        Input('datatable-id', "derived_virtual_data"),
        Input('datatable-id', "derived_virtual_selected_rows")
    ]
)
def update_map(viewData, index):
    if viewData is None or index is None or len(index) == 0:
        return html.P("No location selected")

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

# Run app
app.run_server()


Dash app running on https://matchcanal-axiomorigami-3000.codio.io/proxy/8050/
